# Test notebook

In [1]:
import requests
import os

# load the correct host name from the environment if defined
HOSTNAME = os.getenv('API_HOSTNAME', 'localhost')
PORT = 443
BASE_URL = f'https://{HOSTNAME}:{PORT}'

YOUR_USERNAME = 'mario.rossi'# change this, until topic 7 gives you real credentials anything will work
YOUR_PASSWORD = 'password1'# change this, until topic 7 gives you real credentials anything will work

In [2]:
AUTH_URL = f'{BASE_URL}/user/login'

auth_token = None

def get_token(url, username, password):
    headers = {
        'username': username,
        'password': password
    }  
    response = requests.post(url, headers=headers, verify=False)
    if response.status_code == 200:
        print('You are authenticated!')
        return response.json()
    elif response.status_code == 400:
        print('Wrong credentials')
    elif response.status_code == 500:
        print('Connection error')
    else:
        print('Unknown error')
    return None

def call_with_token(url, token, call_type='GET', header_params=None):
    headers = {} if header_params is None else header_params
    headers['Authorization'] = token
    if call_type == 'GET':
        response = requests.get(url, headers=headers, verify=False)
    elif call_type == 'POST':
        response = requests.post(url, headers=headers, verify=False)
    elif call_type == 'PUT':
        response = requests.put(url, headers=headers, verify=False)
    elif call_type == 'DELETE':
        response = requests.delete(url, headers=headers, verify=False)
    else:
        print('Unknown call type')
        return None, None
    if response.status_code == 500:
        print('Server error: ', response.reason)
        return None, response.status_code
    elif response.status_code == 400:
        print('Bad request: ', response.reason)
        return None, response.status_code
    elif response.status_code == 401:
        print('Unauthorized')
        return None, response.status_code
    elif response.status_code == 200 or response.status_code == 201:
        return response.json(), response.status_code
    else:
        print('Unknown error: ', response.reason)
        return None, response.status_code
    
def call_and_retry(url, token, call_type='GET', header_params=None):
    response, status = call_with_token(url, token, call_type, header_params)
    if status == 401:
        print('Token expired, getting new token')
        token = get_token(AUTH_URL, YOUR_USERNAME, YOUR_PASSWORD)
        global auth_token
        auth_token = token
        if token is None:
            print('Failed to get new token')
            return None, 401
        return call_and_retry(url, token, call_type, header_params)
    return response, status

## Get an authentication token

In [3]:
auth_token = get_token(AUTH_URL, YOUR_USERNAME, YOUR_PASSWORD)
print('Token: ', auth_token)

You are authenticated!
Token:  Bearer token6493


c:\Users\andre\anaconda3\envs\HLT\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Get machine list

When you call an API method, you need the authentication token.
The token may be expired if some time has passed since you got it, so you should get a new one before calling the API if you get an authentication error.

In [14]:
#get machine list
MACHINE_URL = f'{BASE_URL}/machine'
machines, status = call_and_retry(MACHINE_URL, auth_token)
if status == 200:
    print('Machines: ', machines)
else:
    print('Failed to get machines')

Machines:  [{'factory': 'Unknown', 'id': 1, 'machineType': 'Large Capacity Cutting Machine', 'name': 'Large Capacity Cutting Machine 1', 'productionLine': 1, 'status': 'Down'}, {'factory': 'Unknown', 'id': 2, 'machineType': 'Riveting Machine', 'name': 'Riveting Machine', 'productionLine': 0, 'status': 'Down'}, {'factory': 'Unknown', 'id': 3, 'machineType': 'Medium Capacity Cutting Machine', 'name': 'Medium Capacity Cutting Machine 1', 'productionLine': 1, 'status': 'Down'}, {'factory': 'Unknown', 'id': 4, 'machineType': 'Laser Cutter', 'name': 'Laser Cutter', 'productionLine': 0, 'status': 'Down'}, {'factory': 'Unknown', 'id': 5, 'machineType': 'Large Capacity Cutting Machine', 'name': 'Large Capacity Cutting Machine 2', 'productionLine': 2, 'status': 'Down'}, {'factory': 'Unknown', 'id': 6, 'machineType': 'Medium Capacity Cutting Machine', 'name': 'Medium Capacity Cutting Machine 2', 'productionLine': 2, 'status': 'Down'}, {'factory': 'Unknown', 'id': 7, 'machineType': 'Testing Machin

c:\Users\andre\anaconda3\envs\HLT\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Add an alert

In [15]:
ENDPOINT_URL = f'{BASE_URL}/alert'
HEADERS = {
    "timestamp": "2023-10-01 08:00:00",
    "machineID": "1",
    "severity": "low",
    "description": "This is a test alert",
    "KPI": "sum(Temperature) > 300"
}
new_id, status = call_and_retry(ENDPOINT_URL, auth_token, call_type='POST', header_params=HEADERS)
if status == 201:
    print('Alert added: ', new_id)
else:
    print('Failed to add alerts')

c:\Users\andre\anaconda3\envs\HLT\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Unknown error:  CREATED
Alert added:  None


In [16]:
#get the new alert
ALERT_URL = f'{BASE_URL}/alert/id/{new_id}'
alert, status = call_and_retry(ALERT_URL, auth_token)
if status == 200:
    print('Alert: ', alert)
else:
    print('Failed to get alert')

Unknown error:  NOT FOUND
Failed to get alert


c:\Users\andre\anaconda3\envs\HLT\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [4]:
import pickle

In [5]:
alert_file = 'saved_alerts.pkl'
# read
if os.path.exists(alert_file):
    with open(alert_file, 'rb') as f:
        alerts = pickle.load(f)

In [ ]:
ENDPOINT_URL = f'{BASE_URL}/alert'

for alert in alerts:
    new_id, status = call_and_retry(ENDPOINT_URL, auth_token, call_type='POST', header_params=alert)
    if status == 201:
        print('Alert added: ', new_id)
    else:
        print('Failed to add alerts')

In [10]:
import pandas as pd

In [8]:
alerts

[{'timestamp': '2024-09-19 00:00:00',
  'machineID': '2',
  'severity': 'high',
  'description': 'A positive trend drift was detected during the last 7 samples, with respect to the previous 28 samples',
  'KPI': 'consumption_working'},
 {'timestamp': '2024-09-20 00:00:00',
  'machineID': '2',
  'severity': 'high',
  'description': 'A positive trend drift was detected during the last 7 samples, with respect to the previous 28 samples',
  'KPI': 'consumption_working'},
 {'timestamp': '2024-09-21 00:00:00',
  'machineID': '2',
  'severity': 'high',
  'description': 'A positive trend drift was detected during the last 7 samples, with respect to the previous 28 samples',
  'KPI': 'consumption_working'},
 {'timestamp': '2024-09-22 00:00:00',
  'machineID': '2',
  'severity': 'high',
  'description': 'A positive trend drift was detected during the last 7 samples, with respect to the previous 28 samples',
  'KPI': 'consumption_working'},
 {'timestamp': '2024-09-23 00:00:00',
  'machineID': '2'

In [12]:
# load alerts into a pandas dataframe
df = pd.DataFrame(alerts)
df.drop_duplicates(inplace=True)

In [13]:
df

,timestamp,machineID,severity,description,KPI
0,2024-09-19 00:00:00,2,high,A positive trend drift was detected during the...,consumption_working
1,2024-09-20 00:00:00,2,high,A positive trend drift was detected during the...,consumption_working
2,2024-09-21 00:00:00,2,high,A positive trend drift was detected during the...,consumption_working
3,2024-09-22 00:00:00,2,high,A positive trend drift was detected during the...,consumption_working
4,2024-09-23 00:00:00,2,high,A positive trend drift was detected during the...,consumption_working
...,...,...,...,...,...
11954,2024-09-22 00:00:00,4,medium,anomaly detected in historical data,working_time
11955,2024-09-29 00:00:00,4,medium,anomaly detected in historical data,working_time
11956,2024-10-03 00:00:00,4,medium,anomaly detected in historical data,working_time
11957,2024-10-04 00:00:00,4,medium,anomaly detected in historical data,working_time


In [15]:
ENDPOINT_URL = f'{BASE_URL}/alert'

In [ ]:
# insert all alerts into the database
for index, row in df.iterrows():
    HEADERS = {
        "timestamp": row['timestamp'],
        "machineID": row['machineID'],
        "severity": row['severity'],
        "description": row['description'],
        "KPI": row['KPI']
    }
    new_id, status = call_and_retry(ENDPOINT_URL, auth_token, call_type='POST', header_params=HEADERS)
    if status == 201:
        print('Alert added: ', new_id)
    else:
        print('Failed to add alerts')

In [18]:
preprocessed_data_file = 'data_features.pkl'
# read
if os.path.exists(preprocessed_data_file):
    with open(preprocessed_data_file, 'rb') as f:
        data = pickle.load(f)
data

,machineid,kpi,aggregation_type,timestamp,trend_drift,value,imputation,anomaly,next_days_predictions,confidence_interval_lower,confidence_interval_upper
0,3,cycles,sum,2024-03-01 00:00:00,0.0,9126.0,False,False,None,None,None
1,3,cycles,sum,2024-03-02 00:00:00,0.0,2.0,False,False,None,None,None
2,3,cycles,sum,2024-03-03 00:00:00,0.0,0.0,False,False,None,None,None
3,3,cycles,sum,2024-03-04 00:00:00,0.0,61182.0,False,False,None,None,None
4,3,cycles,sum,2024-03-05 00:00:00,0.0,894.0,False,False,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
52653,4,consumption_idle,max,2024-10-15 00:00:00,0.0,0.0,False,False,None,None,None
52654,4,consumption_idle,max,2024-10-16 00:00:00,0.0,0.0,False,False,None,None,None
52655,4,consumption_idle,max,2024-10-17 00:00:00,0.0,0.0,False,False,None,None,None
52656,4,consumption_idle,max,2024-10-18 00:00:00,0.0,0.0,False,False,None,None,None


In [20]:
data[data['machineid'] == 2]['kpi'].unique()

array(['consumption_working', 'average_cycle_time', 'consumption_idle',
       'working_time', 'offline_time', 'cost_working', 'good_cycles',
       'consumption', 'bad_cycles', 'idle_time', 'cost_idle', 'cycles',
       'power', 'cost'], dtype=object)

In [24]:
with open('../../Storage/smart_app_data.pkl', 'rb') as f:
    sensor_data = pd.read_pickle(f)

In [26]:
MACHINE_NAME_TO_ID = {
    "Large Capacity Cutting Machine 1": 1,
    "Riveting Machine" : 2,
    "Medium Capacity Cutting Machine 1": 3,
    "Laser Cutter": 4,
    "Large Capacity Cutting Machine 2": 5,
    "Medium Capacity Cutting Machine 2": 6,
    "Testing Machine 1": 7,
    "Testing Machine 2": 8,
    "Low Capacity Cutting Machine 1": 9,
    "Medium Capacity Cutting Machine 3": 10,
    "Assembly Machine 1": 11,
    "Laser Welding Machine 1": 12,
    "Assembly Machine 2": 13,
    "Assembly Machine 3": 14,
    "Laser Welding Machine 2": 15,
    "Testing Machine 3": 16
}

In [29]:
sensor_data['machineid'] = sensor_data['name'].apply(lambda x: MACHINE_NAME_TO_ID[x])
sensor_data.drop(columns=['asset_id'], inplace=True)

In [32]:
sensor_data.drop(columns=['name'], inplace=True)

In [31]:
# rename columns
sensor_data.rename(columns={'time': 'timestamp'}, inplace=True)

In [35]:
# change timestamp to datetime from yyyy-mm-ddThh:mm:ssZ to yyyy-mm-dd hh:mm:ss
sensor_data['timestamp'] = sensor_data['timestamp'].apply(lambda x: x.replace('T', ' ').replace('Z', ''))

In [37]:
sensor_data['imputation'] = False
sensor_data['anomaly'] = False
sensor_data['next_days_predictions'] = None
sensor_data['confidence_interval_lower'] = None
sensor_data['confidence_interval_upper'] = None
sensor_data['trend drift'] = 0.0

In [38]:
sensor_data

,timestamp,kpi,sum,avg,min,max,machineid,imputation,anomaly,next_days_predictions,confidence_interval_lower,confidence_interval_upper,trend drift
0,2024-03-01 00:00:00,working_time,0.000000,0.000000,0.0,0.000000,1,False,False,None,None,None,0.0
1,2024-03-01 00:00:00,idle_time,0.000000,0.000000,0.0,0.000000,1,False,False,None,None,None,0.0
2,2024-03-01 00:00:00,offline_time,0.000000,0.000000,0.0,0.000000,1,False,False,None,None,None,0.0
3,2024-03-01 00:00:00,consumption,0.066106,0.002321,0.0,0.066106,1,False,False,None,None,None,0.0
4,2024-03-01 00:00:00,power,NaN,0.003673,0.0,0.012801,1,False,False,None,None,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96915,2024-10-19 00:00:00,cost_idle,0.000000,0.000000,0.0,0.000000,16,False,False,None,None,None,0.0
96924,2024-10-19 00:00:00,cycles,0.000000,0.000000,0.0,0.000000,16,False,False,None,None,None,0.0
96925,2024-10-19 00:00:00,good_cycles,0.000000,0.000000,0.0,0.000000,16,False,False,None,None,None,0.0
96926,2024-10-19 00:00:00,bad_cycles,0.000000,0.000000,0.0,0.000000,16,False,False,None,None,None,0.0


In [40]:
sensor_data_sum = sensor_data.drop(columns=['avg', 'min', 'max'])
sensor_data_sum.rename(columns={'sum': 'value'}, inplace=True)
sensor_data_sum['aggregation_type'] = 'sum'
sensor_data_max = sensor_data.drop(columns=['avg', 'min', 'sum'])
sensor_data_max.rename(columns={'max': 'value'}, inplace=True)
sensor_data_max['aggregation_type'] = 'max'
sensor_data_min = sensor_data.drop(columns=['avg', 'max', 'sum'])
sensor_data_min.rename(columns={'min': 'value'}, inplace=True)
sensor_data_min['aggregation_type'] = 'min'
sensor_data_avg = sensor_data.drop(columns=['min', 'max', 'sum'])
sensor_data_avg.rename(columns={'avg': 'value'}, inplace=True)
sensor_data_avg['aggregation_type'] = 'avg'
# merge all dataframes
sensor_data = pd.concat([sensor_data_sum, sensor_data_max, sensor_data_min, sensor_data_avg])
sensor_data

,timestamp,kpi,value,machineid,imputation,anomaly,next_days_predictions,confidence_interval_lower,confidence_interval_upper,trend drift,aggregation_type
0,2024-03-01 00:00:00,working_time,0.000000,1,False,False,None,None,None,0.0,sum
1,2024-03-01 00:00:00,idle_time,0.000000,1,False,False,None,None,None,0.0,sum
2,2024-03-01 00:00:00,offline_time,0.000000,1,False,False,None,None,None,0.0,sum
3,2024-03-01 00:00:00,consumption,0.066106,1,False,False,None,None,None,0.0,sum
4,2024-03-01 00:00:00,power,NaN,1,False,False,None,None,None,0.0,sum
...,...,...,...,...,...,...,...,...,...,...,...
96915,2024-10-19 00:00:00,cost_idle,0.000000,16,False,False,None,None,None,0.0,avg
96924,2024-10-19 00:00:00,cycles,0.000000,16,False,False,None,None,None,0.0,avg
96925,2024-10-19 00:00:00,good_cycles,0.000000,16,False,False,None,None,None,0.0,avg
96926,2024-10-19 00:00:00,bad_cycles,0.000000,16,False,False,None,None,None,0.0,avg
